In [1]:
from datasets.get_datasets import *
from revision import *
from boostsrl import boostsrl
import random
from sklearn.model_selection import KFold

## Parameters

In [2]:
target = 'athleteplaysforteam'
#test_size = 0.3
small_train_size = 0.3
validation_size = 0.2
revision_threshold = 0.95
max_revision_iterations = 10

## Dataset

In [3]:
[facts, pos, neg] = get_nell_dataset(target)

## Background  configuration

In [4]:
bk = ['athleteledsportsteam(+athlete,+sportsteam).',
      'athleteledsportsteam(+athlete,-sportsteam).',
      'athleteledsportsteam(-athlete,+sportsteam).',
      'athleteplaysforteam(+athlete,+sportsteam).',
      'athleteplaysforteam(+athlete,-sportsteam).',
      'athleteplaysforteam(-athlete,+sportsteam).',
      'athleteplaysinleague(+athlete,+sportsleague).',
      'athleteplaysinleague(+athlete,-sportsleague).',
      'athleteplaysinleague(-athlete,+sportsleague).',
      'athleteplayssport(+athlete,+sport).',
      'athleteplayssport(+athlete,-sport).',
      'athleteplayssport(-athlete,+sport).',
      'teamalsoknownas(+sportsteam,+sportsteam).',
      'teamalsoknownas(+sportsteam,-sportsteam).',
      'teamalsoknownas(-sportsteam,+sportsteam).',
      'teamplaysagainstteam(+sportsteam,+sportsteam).',
      'teamplaysagainstteam(+sportsteam,-sportsteam).',
      'teamplaysagainstteam(-sportsteam,+sportsteam).',
      'teamplaysinleague(+sportsteam,+sportsleague).',
      'teamplaysinleague(+sportsteam,-sportsleague).',
      'teamplaysinleague(-sportsteam,+sportsleague).',
      'teamplayssport(+sportsteam,+sport).',
      'teamplayssport(+sportsteam,-sport).',
      'teamplayssport(-sportsteam,+sport).']

background = boostsrl.modes(bk, [target], useStdLogicVariables=False, treeDepth=8, nodeSize=3, numOfClauses=8)

## Sampling examples

In [5]:
# shuffle all examples
random.shuffle(pos)
random.shuffle(neg)

neg = neg[:len(pos)] # balanced

import numpy as np
pos = np.array(pos)
neg = np.array(neg)
small_dataset_aucroc = []
complete_dataset_aucroc = []
revision_dataset_aucroc = []


# separate train and test
kf = KFold(n_splits=10)
for train_index, test_index in kf.split(pos):
    train_pos, test_pos = pos[train_index], pos[test_index]
    train_neg, test_neg = neg[train_index], neg[test_index]

    # learn from scratch in a small dataset
    s_train_pos = train_pos[:int(small_train_size*len(train_pos))]
    s_train_neg = train_neg[:int(small_train_size*len(train_neg))]

    # shuffle all train examples
    random.shuffle(train_pos)
    random.shuffle(train_neg)

    # train set used in revision and validation set
    r_train_pos = train_pos[int(validation_size*len(train_pos)):]
    r_train_neg = train_neg[int(validation_size*len(train_neg)):]
    validation_pos = train_pos[:int(validation_size*len(train_pos))]
    validation_neg = train_neg[:int(validation_size*len(train_neg))]

    print('Total examples')
    print('Positive examples: %s' % len(pos))
    print('Negative examples: %s' % len(neg))
    print('\n')
    print('Train and test sets from total')
    print('Train Positive examples: %s' % len(train_pos))
    print('Train Negative examples: %s' % len(train_neg))
    print('Test Positive examples: %s' % len(test_pos))
    print('Test Negative examples: %s' % len(test_neg))
    print('\n')
    print('Small dataset')
    print('Train Positive examples: %s' % len(s_train_pos))
    print('Train Negative examples: %s' % len(s_train_neg))
    print('\n')
    print('Revision theory train dataset and validation')
    print('Train Positive examples: %s' % len(r_train_pos))
    print('Train Negative examples: %s' % len(r_train_neg))
    print('Validation Positive examples: %s' % len(validation_pos))
    print('Validation Negative examples: %s' % len(validation_neg))
    
    # learning from small dataset
    delete_model_files()
    model = boostsrl.train(background, s_train_pos, s_train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    small_dataset_aucroc.append(t_results['AUC ROC'])
    
    # learning from complete dataset
    delete_model_files()
    model = boostsrl.train(background, train_pos, train_neg, facts)
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    #structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    complete_dataset_aucroc.append(t_results['AUC ROC'])
    
    # theory revision
    total_revision_time = 0
    best_aucroc = 0
    best_structured = None

    print('Performing parameter learning')
    print('******************************************')
    delete_model_files()
    model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=get_refine_file(structured))
    learning_time = model.traintime()
    will = model.get_will_produced_tree()
    structured = model.get_structured_tree().copy()
    results = boostsrl.test(model, validation_pos, validation_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    total_revision_time += learning_time + inference_time
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total learning time: %s seconds' % learning_time)
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)

    best_aucroc = t_results['AUC ROC']
    best_structured = structured.copy()
    save_model_files()

    for i in range(max_revision_iterations):
        print('Refining iteration %s' % str(i+1))
        print('******************************************')
        found_better = False
        for candidate in get_cantidates(best_structured, revision_threshold):
            print('Refining node candidate')
            print('******************************************')
            delete_model_files()
            model = boostsrl.train(background, r_train_pos, r_train_neg, facts, refine=candidate)
            learning_time = model.traintime()
            will = model.get_will_produced_tree()
            results = boostsrl.test(model, validation_pos, validation_neg, facts)
            inference_time = results.testtime()
            t_results = results.summarize_results()
            total_revision_time += learning_time + inference_time
            print('WILL-Produced Tree:')
            print_will_produced_tree(will)
            print('\n')
            print('Results:')
            print(t_results)
            print('\n')
            print('Total learning time: %s seconds' % learning_time)
            print('Total inference time: %s seconds' % inference_time)
            print('AUC ROC: %s' % t_results['AUC ROC'])
            print('\n')
            #print('Tree:')
            #model.tree(0, target, image=True)
            print('******************************************')

            if t_results['AUC ROC'] > best_aucroc:
                found_better = True
                best_aucroc = t_results['AUC ROC']
                best_structured = model.get_structured_tree().copy()
                save_model_files()
        print('Best Tree AUC ROC so far: %s' % best_aucroc)
        print('******************************************\n')
        if found_better == False:
            break

    print('******************************************')
    delete_model_files()
    get_saved_model_files()
    delete_test_files()
    print('Total revision time: %s' % total_revision_time)
    print('Best validation AUC ROC: %s' % best_aucroc)
    will = model.get_will_produced_tree()
    results = boostsrl.test(model, test_pos, test_neg, facts)
    inference_time = results.testtime()
    t_results = results.summarize_results()
    print('WILL-Produced Tree:')
    print_will_produced_tree(will)
    print('\n')
    print('Results:')
    print(t_results)
    print('\n')
    print('Total inference time: %s seconds' % inference_time)
    print('AUC ROC: %s' % t_results['AUC ROC'])
    print('\n')
    #print('Tree:')
    #model.tree(0, target, image=True)
    revision_dataset_aucroc.append(t_results['AUC ROC'])

Total examples
Positive examples: 1458
Negative examples: 1458


Train and test sets from total
Train Positive examples: 1312
Train Negative examples: 1312
Test Positive examples: 146
Test Negative examples: 146


Small dataset
Train Positive examples: 393
Train Negative examples: 393


Revision theory train dataset and validation
Train Positive examples: 1050
Train Negative examples: 1050
Validation Positive examples: 262
Validation Negative examples: 262
WILL-Produced Tree:
% FOR athleteplaysforteam(A, B):
%   if ( athleteledsportsteam(C, B), athleteplaysinleague(C, D), athleteplaysinleague(A, D) )
%   then return 0.7256986039736821;  // std dev = 0,339, 151,000 (wgt'ed) examples reached here.  /* #neg=20 #pos=131 */
%   else if ( athleteplaysinleague(A, E), teamplaysinleague(F, E) )
%   | then return -0.02046378166349359;  // std dev = 0,327, 173,000 (wgt'ed) examples reached here.  /* #neg=152 #pos=21 */
%   | else if ( teamplaysagainstteam(B, G), teamalsoknownas(H, G), teamplaysag

In [6]:
small_dataset_aucroc

[0.80334,
 0.800948,
 0.797148,
 0.77097,
 0.790674,
 0.80212,
 0.787319,
 0.814107,
 0.741046,
 0.800238]

In [7]:
complete_dataset_aucroc

[0.802871,
 0.841527,
 0.844929,
 0.755841,
 0.792785,
 0.835452,
 0.758163,
 0.80578,
 0.780095,
 0.784495]

In [8]:
revision_dataset_aucroc

[0.80334,
 0.795459,
 0.797148,
 0.778758,
 0.786287,
 0.820675,
 0.790228,
 0.813638,
 0.76654,
 0.801332]

In [9]:
print('Small dataset AUC ROC: %s +/- %s' % ((np.array(small_dataset_aucroc)).mean(), (np.array(small_dataset_aucroc)).std()))
print('Complete dataset AUC ROC: %s +/- %s' % ((np.array(complete_dataset_aucroc)).mean(), (np.array(complete_dataset_aucroc)).std()))
print('Theory revision dataset AUC ROC: %s +/- %s' % ((np.array(revision_dataset_aucroc)).mean(), (np.array(revision_dataset_aucroc)).std()))


Small dataset AUC ROC: 0.790791 +/- 0.019853933977929922
Complete dataset AUC ROC: 0.8001938000000001 +/- 0.030708709929920537
Theory revision dataset AUC ROC: 0.7953405000000001 +/- 0.015140535632863206
